In [2]:
import os
print('Process (%s) start...' % os.getpid)
pid = os.fork()
if pid == 0:
    print('I am child process (%s) and my parent is %s' % (os.getpid(), os.getppid()))
else:
    print('I (%s) just created a child process (%s).' % (os.getpid(), pid))

Process (<built-in function getpid>) start...
I am child process (8106) and my parent is 7948


Process (<built-in function getpid>) start...
I (7948) just created a child process (8106).


In [10]:
from multiprocessing import Process
import os

def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

print('Parent process %s.' % os.getpid())
p = Process(target=run_proc, args=('test', ))
print('Child process will start.')
p.start()
p.join()
print('Child process end.')

Child process end.


Run child process test (8462)...


Parent process 7948.
Child process will start.


In [11]:
from multiprocessing import Pool
import os, time, random

def long_time_task(name):
    print('Run task %s (%s)...' % (name, os.getpid()))
    start = time.time()
    time.sleep(random.random() * 3)
    end = time.time()
    print('Task %s runs %0.2f seconds.' % (name, (end - start)))

print('Parent process %s.' % os.getpid())
p = Pool(4)
for i in range(5):
    p.apply_async(long_time_task, args=(i,))
print('Waiting for all subprocesses done...')
p.close()
p.join()
print('All subprocesses done.')

All subprocesses done.


Task 4 runs 1.82 seconds.


Task 0 runs 2.85 seconds.


Task 3 runs 2.45 seconds.


Task 2 runs 2.39 seconds.


...

Run task 0 (8635)...


Run task 2 (8636)...


Run task 1 (8634)...


Run task 3 (8637)...


Parent process 7948.


In [16]:
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


In [17]:
print('$ nslookup')
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print('Exit code:', p.returncode)

Server:		202.96.209.5
Address:	202.96.209.5#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:


Exit code: 0


$ nslookup


In [18]:
from multiprocessing import Process, Queue
import os, time, random

def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

In [19]:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

In [20]:
q = Queue()
pw = Process(target=write, args=(q,))
pr = Process(target=read, args=(q,))
pw.start()
pr.start()
pw.join()
# pr 进程是死循环，无法等待其结束，只能强行终止：
pr.terminate()

Get C from queue.


Put C to queue...


Get B from queue.


Put B to queue...


Get A from queue.


Put A to queue...


Process to write: 9417


Process to read: 9418
